# Makermore - MPL y las capas de activación

## Introducción
* Clase anterior: Modelo de redes neuronales de Bengio et al. 2003 para modelos de lenguajes a nivel de caracteres
* La idea es ir hacia modelos más complejos, como Recurrent Neural Networks, ResNet, WaveNet y eventualmente, modelos basados en mecanismos de atención como los Generative Pretrained Transformers (GPTs).
* Estos modelos, aunque muy expresivos, presentan problemas que los investigadores fueron resolviendo con tiempo y esfuerzo.
  * Modelos muy profundos son difíciles de entrenar (Por qué?)
* La idea de la clase es
  * Optimizar algunas fases del entrenamiento. Kaiming Init.
  * Ganar una intuición sobre el comportamiento de las activaciones de la red y sobre todo, con los gradientes en la etapa de backpropagation. Visualizar las salidas y la información de los gradientes.
  * Pytorchificar el código.
  * Entender por que no es posible entrenar redes muuuuuuuy grandes o profundas.
* Batch Normalization
* ResNet 

## Código inicial...

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
import idna

# def utf8_to_punycode(text: str) -> str:
#     """Encodes a UTF-8 string to its Punycode representation."""
#     return idna.encode(text).decode('ascii')

def punyencode(text: str) -> str:
    """Encodes a UTF-8 string to its Punycode representation, handling spaces by encoding each word separately."""
    
    return " ".join([idna.encode(word).decode('ascii') for word in text.split()])
    
def punydecode(punycode: str) -> str:
    """Decodes a Punycode string back to UTF-8."""
    #return idna.decode(punycode)
    return " ".join([idna.decode(word) for word in punycode.split()])

def process_name(name):
    name = name.lower()
    for n in name.split():
        if len(n) < 2:
            return ''
    try:
        return punyencode(name)
    except:
        #print(f'Cant convert {name}')
        return ''

dataset = open("data/city_names_full.txt", 'r').read().split('\n')
with open('data/city_names_puny.txt', 'w') as f:
    for n in dataset:
        name = process_name(n)
        if name != '':
            f.write(name+'\n')
dataset = open("data/city_names_puny.txt", 'r').read().split('\n')
puny = [x for x in dataset if 'xn--' in x]
nopuny = [x for x in dataset if 'xn--' not in x]
np.random.seed(42)
dataset = [x.item() for x in np.random.choice(nopuny, 100000,replace=False)]

In [ ]:
charset = ['*'] + sorted(list(set([y for x in dataset for y in x])))
ctoi = {c:i for i, c in enumerate(charset)}
itoc = {i:c for i, c in enumerate(charset)}
charset_size = len(charset)

In [ ]:
class Model:
    def __init__(self, charset_size, context_size, emb_size, hidden_size, g=torch.Generator().manual_seed(42)):
        self.charset_size = charset_size
        self.context_size = context_size
        self.emb_size = emb_size
        self.hidden_size = hidden_size
        self.C = torch.randn(self.charset_size, self.emb_size, generator=g)
        self.W1 = torch.randn(self.emb_size*self.context_size, self.hidden_size, generator=g)
        self.b1 = torch.randn(self.hidden_size, generator=g)
        self.W2 = torch.randn(self.hidden_size, self.charset_size, generator=g)
        self.b2 = torch.randn(self.charset_size, generator=g)
        self.parameters = [self.C, self.W1, self.b1, self.W2, self.b2]
        for p in self.parameters:
            p.requires_grad = True
    
    def __call__(self, x):
        self.emb = self.C[x]
        self.embcat = self.emb.view(-1, self.emb_size*self.context_size)
        self.preact = self.embcat @ self.W1 + self.b1
        self.act = torch.tanh(self.preact)
        self.logits = self.act @ self.W2 + self.b2
        return self.logits

    def count_parameters(self):
        return sum([p.nelement() for p in self.parameters])

    def zero_grad(self):
        for p in self.parameters:
            p.grad = None

    def requieres_grad(self):
        for p in self.parameters:
            p.requieres_grad = True
 
    def sample(self, nsamples, g=torch.Generator().manual_seed(42)):
        samples = []
        for n in range(nsamples):
            context = [0] * context_size
            out = []
            while True:
                logits = self(torch.tensor(context))
                probs = F.softmax(logits, dim=1)
                idx = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
                context = context[1:] + [idx]
                out.append(itoc[idx])
                if idx == 0:
                    samples.append(''.join(out[:-1]))
                    break
        return samples

def nll(logits, Y):
    return F.cross_entropy(logits, Y)

In [ ]:
def build_dataset(dataset: list):
    X, Y  = [], []
    for d in dataset:
        example = list(d) + ['*']
        context = [0] * context_size
        for c in example:
            X.append(context)
            Y.append(ctoi[c])
            context = context[1:] + [ctoi[c]] 
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

# build the dataset
context_size = 3
np.random.seed(42)
np.random.shuffle(dataset)
n1 = int(.8 * len(dataset))  # límite para el 80% del dataset
n2 = int(.9 * len(dataset))  # límite para el 90% del dataset
Xtr, Ytr = build_dataset(dataset[:n1])    # 80%
Xva, Yva = build_dataset(dataset[n1:n2])  # 10%
Xte, Yte = build_dataset(dataset[n2:])    # 10%

In [ ]:
emb_size = 10
n_hidden = 256
model = Model(charset_size, context_size, emb_size, n_hidden, torch.Generator().manual_seed(42))
model.count_parameters()

In [ ]:
batch_size = 32

steps = 200000
losses = []
for i in range(steps):
    # 1. Forwards pass
    idx = torch.randint(0, len(Xtr), (batch_size, ))
    logits = model(Xtr[idx])
    
    # 2. loss
    loss = nll(logits, Ytr[idx])
    losses.append(loss.item())
    
    # 3. zero grad
    for p in model.parameters:
        p.grad = None
    
    # 4. backward pass
    loss.backward()
    
    # 5. update
    lr = 0.1 if i < 100000 else 0.01 # learing rate decay
    for p in model.parameters:
        p.data -= p.grad * lr
    if i%10000 == 0:
        print(f'epoch {i}/{steps} loss: {loss.item():0.4f}')
    #break    
print(f'Train loss {nll(model(Xtr), Ytr).item()}')
print(f'Vtion loss {nll(model(Xva), Yva).item()}')

In [ ]:
plt.plot(losses)

In [ ]:
model.sample(10)

## Inicialización de la red para mejorar el loss en los primeros ciclos de entrenamiento

## Visualizando la saturación de la tanh

```python
    def tanh(self):
        x = self.data
        t = (np.e ** (2*x) - 1)/(np.e ** (2*x) + 1)
        out = Value(t, _children=(self, ), _op="tanh")

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out
```

## Arreglando la saturación de la tanh con Kaiming Init
De donde vienen esos números mágicos que multiplican a los pesos Ws y bs?

https://arxiv.org/abs/1502.01852

https://pytorch.org/docs/stable/nn.init.html

## Agregando capas al modelo (PyTorchificando el código)

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(model.layers[-3].out.abs() > .97, cmap='grey', interpolation='nearest')

## Visualizando estadísticas de las capas de activación y los gradientes

# MLPs con Batch Normalization

## Volviendo a nuestro modelo sin modularizar
Para entender en más profundidad los conceptos relacionados con Batch Normalization, volvamos a nuestro modelo sin pytorchificar.

Si queremos que la entrada de nuestra función de activación reciba valores con distribución $\mathbb{N}(0, 1)$, bien podríamos simplemente normalizar `preact`. Como todas las operaciones son diferenciables, es posible hacerlo sin muchos problemas 🤯

https://arxiv.org/pdf/1502.03167

## Pytorchificando la Batch Norm Layer

# ResNet Walkthtough
* https://arxiv.org/pdf/1512.03385
* https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py

# Conclusiones
* Las redes con muchas capas son difíciles de entrenar debido a los problemas del desvanecimiento y explosión de los gradientes.
* La correcta inicialización de la red, no garantiza que los gradientes se desvanezcan o exploten durante la etapa de entrenamiento.
* Cuanto más profunda la red, más dificil de resolver es el problema.
* La normalización por lotes previene estos problemas, normalizando la salida de la capa lineal para no saturar las capa Tanh que sigue.
* La normalización por lotes trae aparejado un costo.
  * Los ejemplos de un batch de entrenamiento quedan asociados matemáticamente.
  * La activación para un ejemplo tiene *jitter*. Vibra dependiendo de los otros ejemplos del batch. Esto resulta ser bueno para el entrenamiento, pero no deja de ser raro y puede tener efectos indeseados y difíciles de debuguear, si uno no sabe bien lo que está pasando. 
  * El modelo deja de poder procesar elementos individuales.
  * Para esto se debe o bien agregar una etapa de calibración para calcular la mean y std del training set o calcular la running mean y std durante el entrenamiento, agregando dos modos posibles a la capa de BatchNorm (training o inference).

# Ejercicios
* Comprobar que el mean y std del training set es compatible con el running mean y std calculado durante el entrenemiento de la capa de BatchNorm1d
* Reimplementar el modelo usando las funciones `nn.Linear()` y `nn.BatchNorm1d()` de PyTorch. 